In [51]:
import pandas as pd
import requests
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import yfinance as yfin
import pandas_ta as ta
import datetime as dt

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [93]:
class EMA_cross_SMA(Strategy):  

        take_prof_ratio = 1.5

        ema = 14
        sma = 20
        atr = 14

        def init(self):     
            self.EMA = self.I(ta.ema, pd.Series(self.data.Close), self.ema)
            self.SMA = self.I(ta.sma, pd.Series(self.data.Close), self.ema)
            self.ATR = self.I(ta.atr, pd.Series(self.data.High), pd.Series(self.data.Low), pd.Series(self.data.Close))
        def next(self):
            price = self.data.Close[-1]
            sl = self.ATR[-1]
            if crossover(self.EMA, self.SMA):
                if self.position.is_short:
                    self.position.close()
                self.buy(sl=(price - sl), tp = (price + (self.take_prof_ratio * sl)))
            elif crossover(self.SMA, self.EMA):
                if self.position.is_long:
                    self.position.close()
                self.sell(sl=(price + sl), tp = (price - (self.take_prof_ratio * sl)))


symbols = ["GBPJPY", "GBPAUD", "GBPCAD", "GBPNZD", "USDJPY", "USDCAD"]
collected_data = {}
collected_data["params"] = {}
collected_data["returns"] = {}

count = 1
for symbol in symbols:
    
    print("Loading data for " + symbol)    
    
    csv_name = "DataSets For Opt Params/" + symbol + "_H1_1Y.csv"
    data = pd.read_csv(csv_name)
    all_rows = []
    for i, row in data.iterrows():
        new_row = str(row.values[0]).split("\t")
        new_row = new_row[:-2]
        new_row = (new_row[0] + " " + new_row[1], new_row[2:])
        all_rows.append(new_row)

    all_dates = [val[0] for val in all_rows]
    all_OHLCV = [val[1] for val in all_rows]
    data = pd.DataFrame(data = all_OHLCV, index = all_dates, columns = ["Open", "High", "Low", "Close", "Volume"])
    data = data.astype(float)
    data.index = pd.to_datetime(data.index)

    bt = Backtest(data, EMA_cross_SMA, cash = 20000, margin = 0.033)

    print("-----------------------------------------------")
    print("Optimizing parameters for " + symbol)
    
    stat, hm = bt.optimize(ema = range(5, 20, 1),
                           sma = range(10, 30, 1),
                           constraint = lambda params: params["ema"] < params["sma"],
                           maximize = "Return [%]",
                           return_heatmap = True)

    if symbol == "GBPCAD":
        print(stat)
        bt.plot(plot_return=True)
        
    collected_data["params"][symbol] = { "EMA" : hm.sort_values().iloc[-1:].index[0][0], "SMA" :  hm.sort_values().iloc[-1:].index[0][1] }
    
    collected_data["returns"][symbol] = stat["Return [%]"]
    
    print(f"Optimal parameters for {symbol} added to data")
    print("-----------------------------------------------")

Loading data for GBPJPY
-----------------------------------------------
Optimizing parameters for GBPJPY


  0%|          | 0/13 [00:00<?, ?it/s]

Optimal parameters for GBPJPY added to data
-----------------------------------------------
Loading data for GBPAUD
-----------------------------------------------
Optimizing parameters for GBPAUD


  0%|          | 0/13 [00:00<?, ?it/s]

Optimal parameters for GBPAUD added to data
-----------------------------------------------
Loading data for GBPCAD
-----------------------------------------------
Optimizing parameters for GBPCAD


...

Optimal parameters for GBPNZD added to data
-----------------------------------------------
Loading data for USDJPY
-----------------------------------------------
Optimizing parameters for USDJPY


  0%|          | 0/13 [00:00<?, ?it/s]

Optimal parameters for USDJPY added to data
-----------------------------------------------
Loading data for USDCAD
-----------------------------------------------
Optimizing parameters for USDCAD


  0%|          | 0/13 [00:00<?, ?it/s]

Optimal parameters for USDCAD added to data
-----------------------------------------------


In [91]:
collected_data

{'params': {'GBPJPY': {'EMA': 19, 'SMA': 29},
  'GBPAUD': {'EMA': 19, 'SMA': 29},
  'GBPCAD': {'EMA': 12, 'SMA': 17},
  'GBPNZD': {'EMA': 19, 'SMA': 29},
  'USDJPY': {'EMA': 19, 'SMA': 29},
  'USDCAD': {'EMA': 19, 'SMA': 29}},
 'returns': {'GBPJPY': 182.0394097607495,
  'GBPAUD': 59.71900482551289,
  'GBPCAD': 899.4517496511802,
  'GBPNZD': 0.3731850824845969,
  'USDJPY': 86.72264531948247,
  'USDCAD': 78.00597280777826}}

In [90]:
total_returns = 0
for v in collected_data["returns"].values():
    total_returns += v

total_returns

1306.311967447188